# Data Profiler Notebook

## Setup

### Import relevant modules

In [ ]:
import pandas
import numpy
import random
import datetime
import math
import json
import seaborn
import matplotlib.pyplot as plt
from functools import reduce

### Set seaborn grid and figuresize

In [ ]:
seaborn.set(rc = {"figure.figsize":(16,12)})
seaborn.set_style("whitegrid")

### Grab current timestamp

In [ ]:
# get timestamp
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d") + "_"

print("Timestamp set to: {}".format(timestamp))

## Settings
- Fill in the parameters within Settings subsections and run all cells to generate report

#### Import data settings
- First import your data into a pandas dataframe (mandatory)
- You can use `pandas`' `read_csv()`, `read_excel()`, `read_json()` methods to read in data

In [ ]:
data_path = "/users/danielcorcoran/desktop/\
github_repos/python_nb_visualization/\
seaborn_official_datasets/iris.csv"

data = pandas.read_csv(data_path)

print("Dataframe has {} features and {} records".format(data.shape[1], data.shape[0]))

In [ ]:
print("Dataframe feature names: {}".format(list(data.columns)))

#### Export settings
- Set your `exportpath`. The profiler will export the .csv/.json report as well as visualization .png/.svgs to this folder.

In [ ]:
exportpath = "/users/danielcorcoran/desktop/github_repos/python_nb_data_profiling/exports/"

print("Export path set to '{}'".format(exportpath))

#### Report Settings 
- Set report width and spacing chars (optional)
- `report_width` is the width in chars of the report printed out in step `1.7 Data Profile Results`
- `header_spacing_char` is the char used to pad the report column headers eg. "##### age (1/12) #####" for "#"
- `attribute_spacing_char` is the char used to pad the attribute value pairing eg. "mean:___34" for "_"

In [ ]:
report_width = 80
just_width = math.floor(report_width / 2)
attribute_spacing_char = " "
header_spacing_char = "~"

print("Report width set to {}".format(report_width))
print("Attribute spacing char set to '{}'".format(attribute_spacing_char))
print("Header spacing char set to '{}'".format(header_spacing_char))

#### Visualization Settings
- `visualize_dataset` if this is set to True, the dataset will be visualized using `seaborn`'s `.pairplot()` and `heatmap()`
- `deal_with_nulls` set this to 'mean' to imputate missing values, or 'remove' to drop them entirely.
- `palette` seaborn palette. Visit https://seaborn.pydata.org/tutorial/color_palettes.html for more info.
- `nunique_range` Categorical features with unique items between this range will be visualized in the `.pairplot()`
- `features_to_exclude` A list of features to exclude from visualization data.
- `identity_type` Set this to 'kde' for kernel density or 'hist' for histogram.
- `non_identity_type` Set this to 'scatter' for scatterplot or 'reg' for regression.

In [ ]:
visualize_dataset = True
features_to_exclude = ['x', 'y', 'z']
deal_with_nulls = "mean"
nunique_range = [2,10]
palette = "Dark2"
identity_type = 'hist'
non_identity_type = 'scatter'

## Extract Data Profiles

### Get Statistical Summary

In [ ]:
#call pandas describe method on dataframe
describe = data.describe(include = "all")

#transpose
describe_transposed = describe.T

#reset_index, moving the column names into a new series
describe_final = describe_transposed.reset_index()

In [ ]:
describe_final

### Get Data Types

In [ ]:
# get datatypes of each feature
dtypes = data.dtypes

In [ ]:
#convert to dataframe and rename a column
dtypes_dataframe = pandas.DataFrame(dtypes, columns = ["data_type"])

In [ ]:
# reset index, moving the column names into a new series
dtypes_final = dtypes_dataframe.reset_index()

In [ ]:
# preview
dtypes_final.head()

### Get Null composition

In [ ]:
#use notnull() and isnull() methods combined with sum() to
#get null composition of dataset
filled_count_series = data.notnull().sum().astype(int)
null_count_series = data.isnull().sum().astype(int)

#get amount of rows in dataset
totalcount = data.shape[0]

In [ ]:
#create percentage series based on series above
percent_null =  null_count_series / totalcount
percent_filled = filled_count_series / totalcount

In [ ]:
percent_null = percent_null.reset_index().rename(columns = {0:"null_percent"})
percent_filled = percent_filled.reset_index().rename(columns = {0:"non_null_percent"})
filled_count_series = filled_count_series.reset_index().rename(columns = {0:"non_null_counts"})
null_count_series = null_count_series.reset_index().rename(columns = {0:"null_counts"})

In [ ]:
null_info_dataframe = reduce(lambda left,right: pandas.merge(left,right,on='index'), [percent_null,
                                                                                 percent_filled,
                                                                                 filled_count_series,
                                                                                 null_count_series])

In [ ]:
null_info_dataframe.head()


### Merge all

In [ ]:
merged = reduce(
    lambda left, right: pandas.merge(left, right, on='index', how="left"),
    [null_info_dataframe, dtypes_final, describe_final])

### Cleanup: Drop count column

In [ ]:
merged["row_count"] = totalcount

merged.drop(["count"], axis = 1, inplace = True)

In [ ]:
merged.head()

### Cleanup: Calculate unique values for numerical features

In [ ]:
# Fill in gaps in unique column with nunique method

for index in range(merged.shape[0]):
    
    unique_value = merged.loc[index, "unique"]
    
    if numpy.isnan(unique_value):
        
        feature_name = merged.loc[index, "index"]
        
        
        number_of_uniques = data[feature_name].nunique()
        merged.loc[index, "unique"] = number_of_uniques
        print("Feature {} unique values being calculated and adjusted to {}".format(feature_name, number_of_uniques))

### Cleanup: Change types

In [ ]:
# Changing type of datatype column to string
merged["data_type"] = merged["data_type"].astype(str)

### Cleanup: Round values

In [ ]:
unique_datatypes = list(merged["data_type"].unique())

print("Unique datatypes in this dataset are: {}".format(unique_datatypes))

if "float64" in unique_datatypes or "int64" in unique_datatypes:
    
    round_values = True
    rounded_places = 4
    
    for column in ["mean", "std", "min", "25%", "50%", "75%", "max"]:
        
        merged[column] = merged[column].astype(float).round(rounded_places)
else:
    round_values = False

### Create Data Profile Dictionary

In [ ]:
profile_dict = {}

In [ ]:
# Store list of features relevant to dictionary
merged_attributes = list(merged.columns)
merged_attributes.remove("index")

# Iterate through merged records
for index in range(merged.shape[0]):
    
    # Store column name
    column = merged.loc[index, "index"]
    
    # Create empty list, to store sub dictionaries in
    profile_dict[column] = []
    
    # For each attribute in merged_attributes list do this
    for attr in merged_attributes:
        
        # Locate the value of the attribuet for a given record
        value = merged.loc[index, attr]
        
        # Append dictionary to list within main dictionary
        profile_dict[column].append({attr: value})

## Export results as .json and .csv

### Helper Function to convert invalid json values

In [ ]:
# will try to convert a value to a float, followed by string,
# if the float conversion fails

def convert_invalid_values(value):
    
    try:
        new_value = float(value)
    except:
        new_value = str(value)

    return new_value

### Export as json

In [ ]:
with open(exportpath + "profile.json", "w") as fileobj:
    # for errors I want to attempt to convert the value to a float before str using
    # the function defined above
    json.dump(profile_dict, fileobj, default = convert_invalid_values)
    
fileobj.close()

### Export as csv

In [ ]:
merged.to_csv(exportpath + "profile.csv", index = False)

## Data Profile Results 

In [ ]:
total_keys = len(profile_dict.keys())
column_number = 0

In [ ]:
for key in profile_dict.keys():
    
    # Increment column number
    column_number = column_number + 1
    
    # Calculate column pos
    column_pos = " ({}/{})"
    
    print("\n", (" " + key + " ").center(report_width, header_spacing_char))
    
    sub_dictionary = profile_dict[key]
    for dictionary in sub_dictionary:
        
        keys = list(dictionary.keys())
        attribute = keys[0]
        value = dictionary[attribute]
        
        if "percent" in attribute:
            formatted_value = "{0:.2%}".format(value)
        else:
            formatted_value = str(value)
        
        print(attribute.ljust(just_width, attribute_spacing_char), 
              formatted_value.rjust(just_width, attribute_spacing_char))

## Visualizations
(Optional)

### Pairplot

#### Create subset of data without nulls for visualizing
- Will react to `deal_with_nulls` variable

In [ ]:
d = "this"

if d in ["this is", "that"]:
    print("true")

In [ ]:
list(merged[(merged["data_type"] == "int64") | 
            (merged["data_type"] == "float64")]["index"])

In [ ]:
if visualize_dataset == True:
    
    if deal_with_nulls == "remove":
        
        # Print message
        print("Nulls will be removed from visualization dataset.")
        
        # Remove each record containing any nulls
        data_no_nulls = data.dropna(how = "any", axis = 0)

        print("Shape of original dataset {}"\
              .format(data.shape))
        print("Shape of visualization dataset {}. \n{} Records were removed (containing nulls)."\
              .format(data_no_nulls.shape, data.shape[0] - data_no_nulls.shape[0]))
    else:
        # Print message
        
        print("Nulls will be replaced by the mean of each numeric feature.")
        
        # Find each column of numeric quality and store in list
        columns_to_fill_mean = list(
                merged[(merged["data_type"] == "int64") | 
                (merged["data_type"] == "float64")]["index"])
        
        data_no_nulls = data
        
        # For each column in list fill the dataset with the mean of that column
        for column in columns_to_fill_mean:
            data_no_nulls[column] = data_no_nulls[column].fillna(data_no_nulls[column].mean())

#### Drop columns from visualization dataset
- These features are listed in `features_to_exclude` variable

In [ ]:
data_no_nulls_column_list = list(data_no_nulls.columns)

if visualize_dataset == True:
    for feature in features_to_exclude:
        if feature in data_no_nulls_column_list:
            data_no_nulls.drop([feature], axis = 1, inplace = True)
            print("{} removed from visualization dataset".format(feature))

#### Calculate chosen_categorical_features (features with unique number of items between given range)

In [ ]:
if visualize_dataset == True:

    # Create subset of merged containing data type = object and nunique between range above
    chosen_categorical_features = list(merged[(merged["unique"] >= nunique_range[0]) & 
                               (merged["unique"] <= nunique_range[1]) & 
                               (merged["data_type"] == "object")]["index"])

    print("Categorical features: {}".format(chosen_categorical_features))

    # Remove unwanted features
    for feature in features_to_exclude:
        if feature in chosen_categorical_features:
            chosen_categorical_features.remove(feature)

    # Preview chosen_categorical_features
    print("Categorical features being used in pairplot: {}".format(chosen_categorical_features))

#### Calculate which columns can be used as categories to split float data in pairplots
- Note: code hasn't accounted for datasets without float/int features yet.

#### Convert bool dtypes before visualizing, as this produces errors otherwise 

In [ ]:
for column in data_no_nulls.columns:
    dtype = data_no_nulls[column].dtype
    if dtype == "bool":
        data_no_nulls[column] = data_no_nulls[column].astype(str)

In [ ]:
if visualize_dataset == True:
    
    # If dataset contains at least one categorical feature do this
    if "unique" in merged.columns:

        for column in chosen_categorical_features:

            plt.figure()
            myplot = seaborn.pairplot(data = data_no_nulls, 
                                      kind = non_identity_type,
                                      diag_kind = identity_type,
                                      hue = column,
                                      palette = palette
                                     )

            myplot.fig.suptitle("Pairplot categorized by {}".format(column), 
                         y = 1.03)

            plt.savefig(exportpath + timestamp +  column + "_pairplot.png")

    # If data set has no categorical features (only floats/ints) do this
    else:

        plt.figure()

        myplot = seaborn.pairplot(data = data_no_nulls, 
                                  kind = non_identity_type,
                                  diag_kind = identity_type
                                 )

        myplot.fig.suptitle("Pairplot", 
                     y = 1.03)

        plt.savefig(exportpath + timestamp +  "data_profile_pairplot.png")

### Correlation Heatmap

In [ ]:
seaborn.set(rc = {"figure.figsize":(13,10)})
seaborn.set_style("whitegrid")

In [ ]:
if visualize_dataset == True and round_values == True:
    # Save path
    heatmap_save_path = exportpath + timestamp + "heatmap.png"
    
    # Create correlation Matrix
    correlation_dataframe = data_no_nulls.corr()
    
    # Create mask
    mask = numpy.zeros_like(correlation_dataframe)
    mask[numpy.triu_indices_from(mask)] = True

    # Create heatmap, show and export as .png
    seaborn.heatmap(data = correlation_dataframe,
               cmap = ['#b2182b',
                    '#ef8a62',
                    '#fddbc7',
                    '#f7f7f7',
                    '#d1e5f0',
                    '#67a9cf',
                    '#2166ac'],
               center = 0,
               square = True,
               linewidth = 1,
               mask = mask,
               annot = True).get_figure().savefig(heatmap_save_path)
    
    print("Heatmap saved to '{}'".format(heatmap_save_path))
else:
    print("No heatmap was produced. Dataset contains no numeric features or visualize_dataset variable was set to False.")